# Evaluation NER - échantillon tiré d'une transcription manuelle - segmentation automatique des phrases

## Imports

In [1]:
import re

from bs4 import BeautifulSoup
import re
import spacy
import srsly

## Fonction d'ouverture des fichiers

In [2]:
def open_file(source):
    '''open a file and parsed its content with BeautifulSoup'''
    with open(source, 'r', encoding='utf8') as fh:
        file_content = fh.read()
        
    return file_content

## Création de la vérité de terrain NER

La vérité de terrain NER a été produite à l'aide de [Doccano](https://github.com/doccano/doccano). Le texte utilisé pour constituer une vérité de terrain est identique à celui utilisé pour la prédiction de NER. 

Cette méthodologie de test est plus adapté à l'évaluation d'un modèle entraîné pour une tâche spécifique. Cependant, il a été jugé pertinent de comparer ce qu'il était obtenu avec l'outil de NER brut, et ce qu'on attendrait de cet outil brut **au mieux** : la vérité de terrain. 

Le modèle de langue français utilisé avec spaCy permet de repérer 4 types d'entités : 
* `PER` : personne
* `LOC` : localisation
* `ORG` : organisations
* `MISC` : une entité générique indiquant une entité nommée, sans toutefois l'identifier.

Pour annoter le document, le fichier texte de l'échantillon testé a été annoté dans Doccano.

La composition d'une vérité de terrain sur un échantillon en sortie d'HTR brut est particulier, car demandant d'annoter des mots bruités, cela s'apparente, à certains moments, presque à lire une autre langue.

Il a été décidé de se placer au niveau des tokens, et d'annoter si celui-ci aurait dû être détecté. Par exemple, les noms de personne arrivant systématiquement au début d'une rangée dans le document originel. 

Les annotations manuelles respectent deux règles : 

* Les titres de civilités n'ont pas été relevés. 
* Les adresses ont été annotes dans leur forme complète, en incluant donc le mot "rue".

## Traitement de l'échantillon testé

 On charge le [modèle de langue française](https://spacy.io/models/fr#fr_core_news_lg) de spaCy.

On récupère le texte qui servira à produire une prédiction pour le NER avec spaCy depuis un fichier texte. 

On essaye de normaliser au minimum en séparant les mots agglutinés à l'aide des majuscules.

In [51]:
with open('../../corpus_test/lectaurep/doc_28_sample/manual_transcription/no_sentence_segmentation/FRAN_0025_0029_L-0.xml') as file:
    sample_eval = BeautifulSoup(file, "lxml-xml")
    
parsed_sample_eval = sample_eval.find_all("Unicode")
transcription_list = [i.text for i in parsed_sample_eval]
transcription_list_preprocessed = [re.sub(r"(\w)([A-Z])", r"\1 \2", i) for i in transcription_list]
parsed_sample_eval = '\n'.join(transcription_list_preprocessed) 

In [52]:
print(parsed_sample_eval)

An 1901 , mois de Février
Dupuis, (par Pierre) à Paris, rue Turgot 4, à Catherine Bizial, sa femme
- d° - (par Mme) s. n. à son mari
Murel (par Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine
Lemaire, dt à Paris, même adresse (Séparation de biens)
Dubosc (ct titre de 380+ de rente 3% au nom  de : Debuisson, Justine Adèle,
Vve de Jules) et autres, à Paris, rue de l’Arcade 20
Rouy (par Edmond Narcisse), dt à Paris, rue Cavalotti 15, et autres, pr toucher &amp; recevoir
Chabrol (par Marie Louise Vatonne, Ve de Marie Eusèbe Maxime) dt à Billancourt, route de 
Versailles, 128, à Eugène Vatrin et Marceline Freling, dt à Billancourt, rue Nationale 31, de 21 303,88
Gauthier (par Marie Alexandrine) à Paris, rue St Ferdinand 2, à ses père et mère
Cusson (par Georges Ernest Léon) dt à Paris, rue Lamarck 144, à son père
Lecrosnier (ct livret de caisse d’Epargne de Paris, de 157,19 au nom de Pierre
Auguste) décédé en son domicile à Paris, rue Descartes 21, le 16 8bre 1900
Let

On passe le texte dans la pipeline NLP.

In [41]:
doc = nlp(sample_eval_splitted)

### Traitement de la vérité de terrain

On récupère la vérité de terrain dans un dictionnaire sous le format suivant :

```
{entities: [(start, end, type)]}
```

On récupère la vérité de terrain dans un dictionnaire sous le format suivant :

```
{entities: [(start, end, type)]}
```

In [42]:
filepath = '../../corpus_test/lectaurep/doc_28_sample/manual_transcription/no_sentence_segmentation/ground_truth_ner/gt_ner.jsonl'
# On utilise le package srsly de spaCy pour transformer un fichier JSONL -format d'export de Doccano- en une liste de dictionnaires
annotations = list(srsly.read_jsonl(filepath))

annotations

[{'id': 9,
  'data': 'An 1901\xa0, mois de Février\nDupuis, (par Pierre) à Paris, rue Turgot 4, à Catherine Bizial, sa femme\n- d° - (par Mme) s. n. à son mari\nMurel (par Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine\nLemaire, dt à Paris, même adresse (Séparation de biens)\nDubosc (ct titre de 380+ de rente 3% au nom  de\xa0: Debuisson, Justine Adèle,\nVve de Jules) et autres, à Paris, rue de l’Arcade 20\nRouy (par Edmond Narcisse), dt à Paris, rue Cavalotti 15, et autres, pr toucher &amp; recevoir\nChabrol (par Marie Louise Vatonne, Ve de Marie Eusèbe Maxime) dt à Billancourt, route de \nVersailles, 128, à Eugène Vatrin et Marceline Freling, dt à Billancourt, rue Nationale 31, de 21 303,88\nGauthier (par Marie Alexandrine) à Paris, rue St Ferdinand 2, à ses père et mère\nCusson (par Georges Ernest Léon) dt à Paris, rue Lamarck 144, à son père\nLecrosnier (ct livret de caisse d’Epargne de Paris, de 157,19 au nom de Pierre\nAuguste) décédé en son domicile à Par

In [59]:
gt_data = {}
for annotation in annotations :
    entities = [(start, end, type_) for start, end, type_ in annotation.get('label')]
    gt_data = {'entities': entities}
    text_gt = annotation.get('data')

gt_data

{'entities': [(0, 8, 'MISC'),
  (39, 45, 'PER'),
  (49, 54, 'LOC'),
  (72, 88, 'PER'),
  (133, 138, 'PER'),
  (144, 162, 'PER'),
  (169, 174, 'LOC'),
  (199, 222, 'PER'),
  (229, 234, 'LOC'),
  (271, 277, 'PER'),
  (321, 330, 'PER'),
  (332, 345, 'PER'),
  (374, 379, 'LOC'),
  (410, 425, 'PER'),
  (433, 438, 'LOC'),
  (495, 502, 'PER'),
  (508, 528, 'PER'),
  (536, 555, 'PER'),
  (562, 573, 'LOC'),
  (604, 617, 'PER'),
  (621, 638, 'PER'),
  (645, 656, 'LOC'),
  (658, 674, 'LOC'),
  (689, 697, 'PER'),
  (703, 720, 'PER'),
  (724, 729, 'LOC'),
  (731, 749, 'LOC'),
  (770, 776, 'PER'),
  (782, 801, 'PER'),
  (808, 813, 'LOC'),
  (843, 853, 'PER'),
  (915, 929, 'PER'),
  (956, 961, 'LOC'),
  (963, 976, 'LOC'),
  (1025, 1036, 'PER'),
  (1059, 1080, 'LOC'),
  (1098, 1104, 'PER'),
  (1110, 1134, 'PER'),
  (1156, 1172, 'PER'),
  (1188, 1197, 'LOC'),
  (1199, 1206, 'LOC'),
  (1213, 1227, 'PER'),
  (1290, 1327, 'PER'),
  (1330, 1348, 'PER'),
  (1361, 1367, 'LOC'),
  (1410, 1417, 'PER'),
  (1423

In [44]:
print(text_gt)

An 1901 , mois de Février
Dupuis, (par Pierre) à Paris, rue Turgot 4, à Catherine Bizial, sa femme
- d° - (par Mme) s. n. à son mari
Murel (par Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine
Lemaire, dt à Paris, même adresse (Séparation de biens)
Dubosc (ct titre de 380+ de rente 3% au nom  de : Debuisson, Justine Adèle,
Vve de Jules) et autres, à Paris, rue de l’Arcade 20
Rouy (par Edmond Narcisse), dt à Paris, rue Cavalotti 15, et autres, pr toucher &amp; recevoir
Chabrol (par Marie Louise Vatonne, Ve de Marie Eusèbe Maxime) dt à Billancourt, route de 
Versailles, 128, à Eugène Vatrin et Marceline Freling, dt à Billancourt, rue Nationale 31, de 21 303,88
Gauthier (par Marie Alexandrine) à Paris, rue St Ferdinand 2, à ses père et mère
Cusson (par Georges Ernest Léon) dt à Paris, rue Lamarck 144, à son père
Lecrosnier (ct livret de caisse d’Epargne de Paris, de 157,19 au nom de Pierre
Auguste) décédé en son domicile à Paris, rue Descartes 21, le 16 8bre 1900
Let

## Vérification de l'alignement entre l'échantillon testé pour la prédiction NER et le texte utilisé pour la vérité de terrain NER

Pour vérifier l'alignement des deux textes utilisés pour la comparaison, on utilise un assert.

In [62]:
assert text_gt == parsed_sample_eval

# Evaluation

On crée un objet Example contenant normalement des informations pour l'entraînement. Il sera utilisé pour stocker le document traité par la pipeline NLP et donc les prédictions de NER, ainsi que la vérité de terrain. 

Voir :
* https://spacy.io/api/example
* https://spacy.io/api/data-formats#dict-input
* https://spacy.io/api/scorer#score_spans

In [64]:
from spacy.training import Example

example = Example.from_dict(doc, gr_data)

/home/hugoscheithauer/anaconda3/envs/ner_spacy/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "An 1901 , mois de Février
Dupuis, (par Pierre) à P..." with entities "[(0, 8, 'MISC'), (39, 45, 'PER'), (49, 54, 'LOC'),...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


On instancie l'objet Scorer.

In [70]:
from spacy.scorer import Scorer

scorer = Scorer()
# On récupère les scores.
scores = scorer.score_spans([example], 'ents')

In [71]:
scores

{'ents_p': 0.6011904761904762,
 'ents_r': 0.6196319018404908,
 'ents_f': 0.6102719033232628,
 'ents_per_type': {'ORG': {'p': 0.42857142857142855,
   'r': 0.42857142857142855,
   'f': 0.42857142857142855},
  'MISC': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'LOC': {'p': 0.5735294117647058,
   'r': 0.5492957746478874,
   'f': 0.5611510791366907},
  'PER': {'p': 0.7564102564102564,
   'r': 0.7023809523809523,
   'f': 0.728395061728395}}}

On peut procéder autrement avec :

In [72]:
from spacy.language import Language

scores_alt = nlp.evaluate([example])

scores_alt

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'pos_acc': None,
 'morph_acc': None,
 'morph_per_feat': None,
 'sents_p': None,
 'sents_r': None,
 'sents_f': None,
 'dep_uas': None,
 'dep_las': None,
 'dep_las_per_type': None,
 'ents_p': 0.6121212121212121,
 'ents_r': 0.6196319018404908,
 'ents_f': 0.6158536585365854,
 'ents_per_type': {'MISC': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'ORG': {'p': 0.42857142857142855,
   'r': 0.42857142857142855,
   'f': 0.42857142857142855},
  'LOC': {'p': 0.5735294117647058,
   'r': 0.5492957746478874,
   'f': 0.5611510791366907},
  'PER': {'p': 0.7763157894736842,
   'r': 0.7023809523809523,
   'f': 0.7374999999999999}},
 'tag_acc': None,
 'lemma_acc': None,
 'speed': 5011.94678110636}

In [73]:
len_gr_entities = len(gr_data['entities'])
len_prediction_entities = len(doc.ents)

In [74]:
print(f"nombre d'entités dans la vérité de terrain : {len_gr_entities} - nombre d'entités dans la prédiction : {len_prediction_entities}")

nombre d'entités dans la vérité de terrain : 168 - nombre d'entités dans la prédiction : 168


# Visualisation de la prédiction NER

In [75]:
from spacy import displacy

html = displacy.render([doc], style='ent', page=True)